In [ ]:
# TODO K Nearest Neighbours
# 
# Load features from file, then apply model
# Try diff params, give accuracies, misclassification rates for each dataset
# Plot confusion matrix, ROC, DET
# Do not use inbuilt library for KNN
# Do on three forms of data -- normal, after PCA, after LDA

In [ ]:
# Take command line args
# feature type - normal, LDA, PCA
# dataset - Image, Synthetic, IsolatedDigits, HandwrittenCharacters

In [ ]:
import numpy as np

In [ ]:
INF = 9999999999

In [ ]:
def euclidean(x,y):
    return np.linalg.norm(x-y)

def cost(v1,v2,angle=False):
    '''
    Computes cost of difference between two vectors
    '''

    if not angle:
        return euclidean(v1,v2)
    else:
        diff = v2-v1
        # diff[np.where(diff>np.pi)] -= 2*np.pi
        # diff[np.where(diff<-np.pi)] += 2*np.pi
        if diff > np.pi:
            diff -= 2*np.pi 
        if diff < -np.pi:
            diff += 2*np.pi
        return np.abs(diff)

def dtw(x, y, angle=False):
    '''
    Computes DTW between sequences of MFCC vectors x and y 
    '''
    
    XLEN = len(x)
    YLEN = len(y)
    
    dp = [[INF for j in range(YLEN+1)] for i in range(XLEN+1)]
    dp[0][0] = 0

    for i in range(1,XLEN+1):
        for j in range(1,YLEN+1):
            cost_here = cost(x[i-1], y[j-1], angle)
            dp[i][j] = cost_here + min([dp[i-1][j], dp[i][j-1], dp[i-1][j-1]])
    
    return dp[-1][-1]

def dtwAngle(x, y, angle=True):
    '''
    Computes DTW between sequences of MFCC vectors x and y 
    '''
    
    XLEN = len(x)
    YLEN = len(y)
    
    dp = [[INF for j in range(YLEN+1)] for i in range(XLEN+1)]
    dp[0][0] = 0

    for i in range(1,XLEN+1):
        for j in range(1,YLEN+1):
            cost_here = cost(x[i-1], y[j-1], angle)
            dp[i][j] = cost_here + min([dp[i-1][j], dp[i][j-1], dp[i-1][j-1]])
    
    return dp[-1][-1]

def knn(train_feats, dev_feats, distfun=euclidean, k=5):
    # Tag each example with its class

    # Find k min distances between test and all train examples
    # Dist function varies -- For synth its direct euclidean
    # For image, find squared distance b/w corresponding blocks, add them up, then rank
    # For digit it's dtw
    # For char it's dtw

    correct = 0
    total = 0

    for dev_cl in sorted(list(dev_feats.keys())):
        for dev_fn in sorted(list(dev_feats[dev_cl].keys())):
            df = dev_feats[dev_cl][dev_fn]
            
            dev_dists = []
            for train_cl in sorted(list(train_feats.keys())):
                for train_fn in sorted(list(train_feats[train_cl].keys())):
                    tf = train_feats[train_cl][train_fn]
                    
                    dev_dists.append([distfun(tf,df), train_cl])
            
            topk = dev_dists.sort()[:k]
            votes = {}
            for cand in topk:
                if cand[1] in votes:
                    votes[cand[1]] += 1
                else:
                    votes[cand[1]] = 1
            
            pred_cl = max(votes, key=votes.get)
            if dev_cl == pred_cl:
                print(f"Passed {dev_fn} Pred class: {pred_cl}")
                correct += 1
            else:
                print(f"Failed {dev_fn} -- Expected {dev_cl}, predicted {pred_cl}")
            
            total += 1
    acc = correct/total
    print("Acc:", acc)